In [1]:
# reference: https://github.com/run-llama/llama_index/blob/main/docs/examples/workflow/rag.ipynb

In [2]:
# setup
import os
from dotenv import find_dotenv, load_dotenv
_ = load_dotenv(find_dotenv())

from llama_index.core.workflow import (
    Context,
    Workflow,
    StartEvent,
    StopEvent,
    step,
)

# event

In [3]:
from llama_index.core.workflow import Event
from llama_index.core.schema import NodeWithScore

class RetrieverEvent(Event):
    """Result of running retrieval"""

    nodes: list[NodeWithScore]


class RerankEvent(Event):
    """Result of running reranking on retrieved nodes"""

    nodes: list[NodeWithScore]

# the workflow

In [4]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

# pip install llama-index-vector-stores-faiss faiss-cpu
import faiss
from llama_index.core import StorageContext
from llama_index.core import load_index_from_storage
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex

from llama_index.core.postprocessor.llm_rerank import LLMRerank

from llama_index.core.response_synthesizers import CompactAndRefine

In [5]:
class RAGWorkflow(Workflow):
    @step
    async def ingest(self, ctx: Context, ev: StartEvent) -> StopEvent | None:
        """document_dir, index_dir"""
        document_dir = ev.get("document_dir")
        index_dir = ev.get("index_dir")
        if not index_dir:
            # 如果沒有 index_dir 這條路就會直接走不通
            # 所以 query 的時候預設是不用給 index_dir 的
            return None
        if not os.path.isdir(index_dir):
            print(f'create index_dir: {index_dir}')
            os.makedirs(index_dir)
        if document_dir:
            d = 1536
            faiss_index = faiss.IndexFlatL2(d)
            vector_store = FaissVectorStore(faiss_index=faiss_index)
            storage_context = StorageContext.from_defaults(
                vector_store=vector_store
            )
            index = VectorStoreIndex.from_documents(
                documents=documents,
                embed_model=embed_model,
                storage_context=storage_context,
            )
            index.storage_context.persist(persist_dir=index_dir)
            print("Index built and persisted to:", index_dir)
        else:
            print(f"load index from: {index_dir}")
            # load index from disk
            vector_store = FaissVectorStore.from_persist_dir(index_dir)
            storage_context = StorageContext.from_defaults(
                vector_store=vector_store, persist_dir=index_dir
            )
            index = load_index_from_storage(storage_context=storage_context)
        return StopEvent(result=index)

    @step
    async def retrieve(
        self, ctx: Context, ev: StartEvent
    ) -> RetrieverEvent | None:
        "Entry point for RAG, triggered by a StartEvent with `query`."
        query = ev.get("query")
        index = ev.get("index")

        if not query:
            return None

        print(f"Query the database with: {query}")

        # store the query in the global context
        await ctx.store.set("query", query)

        # get the index from the global context
        if index is None:
            print("Index is empty, load some documents before querying!")
            return None

        retriever = index.as_retriever(similarity_top_k=8)
        nodes = await retriever.aretrieve(query)
        print(f"Retrieved {len(nodes)} nodes.")
        
        retrieved_nodes = []
        for node in nodes:
            rv = {
                'id': node.id_,
                'text': node.text,
                'score': node.score
            }
            retrieved_nodes.append(rv)
        await ctx.store.set("retrieved_nodes", retrieved_nodes)
        return RetrieverEvent(nodes=nodes)

    @step
    async def rerank(self, ctx: Context, ev: RetrieverEvent) -> RerankEvent:
        # Rerank the nodes
        ranker = LLMRerank(
            choice_batch_size=5, top_n=3, llm=OpenAI(model="gpt-4o-mini")
        )
        print(await ctx.store.get("query", default=None), flush=True)
        new_nodes = ranker.postprocess_nodes(
            ev.nodes, query_str=await ctx.store.get("query", default=None)
        )
        print(f"Reranked nodes to {len(new_nodes)}")
        reranked_nodes = []
        for node in new_nodes:
            rv = {
                'id': node.id_,
                'text': node.text,
                'score': node.score
            }
            reranked_nodes.append(rv)
        await ctx.store.set("reranked_nodes", reranked_nodes)
        return RerankEvent(nodes=new_nodes)

    @step
    async def synthesize(self, ctx: Context, ev: RerankEvent) -> StopEvent:
        """Return a streaming response using reranked nodes."""
        llm = OpenAI(model="gpt-4o-mini")
        summarizer = CompactAndRefine(llm=llm, streaming=False, verbose=True)
        query = await ctx.store.get("query", default=None)
        response = await summarizer.asynthesize(query, nodes=ev.nodes)
        retrieved_nodes = await ctx.store.get("retrieved_nodes", default=None)
        reranked_nodes = await ctx.store.get("reranked_nodes", default=None)
        rv = {
            'query': query,
            'response': response,
            'retrieved_nodes': retrieved_nodes,
            'reranked_nodes': reranked_nodes,
        }
        return StopEvent(result=rv)

# visualize

In [6]:
from llama_index.utils.workflow import draw_all_possible_flows
# Draw all
draw_all_possible_flows(RAGWorkflow, filename="rag_flow_all_self.html")

rag_flow_all_self.html


# run

In [7]:
w = RAGWorkflow()
index_dir = 'storage'
# Ingest the documents
index = await w.run(index_dir=index_dir)

load index from: storage
Loading llama_index.core.storage.kvstore.simple_kvstore from storage/docstore.json.
Loading llama_index.core.storage.kvstore.simple_kvstore from storage/index_store.json.


gio: file:///home/poyuan/workspace/rag30/days/day23/rag_flow_all_self.html: Failed to find default application for content type ‘text/html’
/home/poyuan/miniconda3/envs/rag30/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
result = await w.run(query="什麼是深度學習的醍醐味?", index=index)
print(result['response'].response)

Query the database with: 什麼是深度學習的醍醐味?
Retrieved 8 nodes.
什麼是深度學習的醍醐味?
Reranked nodes to 3
深度學習的醍醐味在於面對模型訓練過程中的焦躁與迷茫，尤其是在等待人工智慧訓練結果和調整參數的過程中。這種不確定性和挑戰感是學習過程的一部分，讓學習者體驗到模型訓練的真實感受。


In [10]:
result['retrieved_nodes']

[{'id': '18773912-c8b5-4fb4-8d4b-cb4e38758012',
  'text': 'Cool上留言區的問題\n那旁聽生\n如果你想要加入NTU Cool的話\n請直接寄信給助教\n只要你寄信給助教說你要旁聽\n我們都可以把你加入NTU Cool\n那怎麼找到NTU Cool上面的留言區呢\n假設你已經有選上這門課的話\n那你就已經在NTU Cool裡面\n那你在NTU Cool裡面點討論\n然後你就可以看到9月12號上課\n即時討論區這一個區域\n然後你可以把你的問題留在這個地方\n如果我有時間的話\n會優先回答在討論區裡面的問題\n那另外一個很重要的訊息要跟大家講的就是\n因為呢\n網路不是非常的穩定\n所以直播是有可能斷線的\n我們剛才測試的時候\n其實直播就斷線了好幾次\n那直播斷線有可能造成的問題就是\n你可能會沒有辦法用同一個連結\n再繼續看直播\n所以假設我們遇到斷線的狀況\n接下來我們的操作是\n會開一個新的直播連結\n然後透過NTU Cool寄給所有有修課的同學\n但是假設你發現直播斷線\n非常重要的一點就是\n請勿驚慌 請勿驚慌',
  'score': 1.8999251127243042},
 {'id': 'f1b8dce2-5b45-4ed8-b44d-0bf6e2306877',
  'text': 'Cool寄給所有有修課的同學\n但是假設你發現直播斷線\n非常重要的一點就是\n請勿驚慌 請勿驚慌 請勿驚慌\n所有的課程統統都有錄影\n所以直播斷線你就去做別的事情\n反正所有的課程都是有錄影的\n好接下來呢\n我們講一下作業的規劃\n我們這門課總共有十講\n每一講都有一個對應的作業\n那我把每一個作業的公告日期\n還有截止日期都放在投影片上面\n那基本上的每一個作業呢\n都是留給大家三週的時間來完成\n那唯一例外的呢\n是前兩個作業\n前兩個作業呢\n我們特別把它的截止日期延後\n延後到10月17號繳交\n因為我知道說這門課\n是很多不同學校的學生一起選修\n那每一個學校加退選的時間都不一樣\n所以大家加選到這門課的時間是不一樣的\n所以前面幾個作業的日期延後\n讓大家比較有時間可以完成\n前面幾個作業\n那另外一個要注意的事情就是\n我們最後一個作業\n作業時的截止日期是明年的1月9號\n那我們要收到

In [9]:
result['reranked_nodes']

[{'id': 'e6955f99-8070-4c73-aa72-c338f2c2c1c2',
  'text': '作業設計成這樣呢\n為什麼作業要跑三個小時呢\n我只允許這個作業三分鐘\n就應該要跑完\n我們有當然可以把這些特別花時間\n需要特別花時間訓練的作業拿掉\n但是我還是選擇在這門課裡面\n保留那一些\n需要一定訓練時間的作業\n因為焦躁的等待人工智慧訓練的結果\n迷茫的調參數\n不知道會不會成功\n這個就是人工智慧的醍醐味\n所以大家需要學習\n在迷茫中前進\n這個就是模型的訓練\n我們特別把這一部分保留在課程裡面\n讓你體驗說\n模型訓練不出來的焦躁\n到底是什麼樣的感覺\n而且我必須要強調啊\n什麼三四個小時的等待訓練時間\n真的不算什麼\n我們過去有很多作業訓練時間都是\n至少三天起跳\n你要至少訓練三天\n你才能夠拿到成績\n那我知道說很多同學在做作業的時候\n往往你都在實現\n最後一天才開始做作業\n但那一種啊\n需要訓練三天以上的模型\n你只在前一天才開始做作業\n你是絕不可能完成的\n這個時候我告訴你\n你唯一可以做的事情\n就是放棄這樣子\n還好我們這堂課裡面呢\n現在是沒有需要訓練一天以上的作業啦\n我們把那種\n特別需要花時間訓練的作業\n還是拿掉了\n只保留了需要訓練三四個小時的作業\n但我只想要強調說\n三四個小時的訓練時間\n真的不算什麼\n如果你要真正用大量的資料\n大規模的訓練模型\n訓練個數週\n其實都是常見的事情\n而這一些\n需要一點訓練時間的作業\n它的定位就像是預防針\n幫助你在未來面對更大的挑戰\n幫助你在未來面對挑戰的時候\n做好心理準備\n另外呢\n鼓勵大家如果有空的話\n可以先看一些線上的錄影進行預習\n那假設你對於生成式AI一無所知的話\n那你可以先看生成式AI導論2024的課程\n那如果2024的課程看完\n你想要進一步了解\n這些AI是怎麼被訓練出來',
  'score': 9.0},
 {'id': 'c9b0c2ac-d7b7-4183-9bcd-6c3d57686b61',
  'text': 'to this course\n這個也是VO3自己生成的\n他產生的影片是帶有聲音的\n那其他中文講課的聲音也通通都是合成的\n我是用Eleven Labs這個軟體合成的\n我是用Eleven La